# **DATA EXPLORATION**
---
This notebook goes through some of the fields we want to clean. We hope to identify some patterns we can use to clean the data.

## **ESTABLISHMENT'S NAME**
---

In [ ]:
import sqlite3
import pandas as pd

# dirty database
db_dirty = sqlite3.connect('../data/dirty_data/dirty_food_inspections.db')
cur = db_dirty.cursor()

# clean database
db_clean = sqlite3.connect('../data/clean_data/food_inspections.db')
cur = db_clean.cursor()

### **OVERVIEW**
In the dataset, the establishment's names have issues with consistency. Many establishments with the same name are spelled differently or have uncessary information attached to it's name.

### **Dirty Data**

The query shows the different forms the establishment **SUBWAY** can be respresented. Some differences include:

* **address information** (e.g. LINCOLN AVE SUBWAY on row 6)

* **number values** (e.g. SUBWAY #38326 on row 10)

* **capitolization differences** (e.g. Lakeview Subway on row 7)


In [ ]:
Q = '''
SELECT estName as Name, COUNT(estName) as 'num of establishments'
FROM Establishments
WHERE Name LIKE '%subway%'
GROUP BY Name
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_dirty)
df.head(15)

### **AFTER**

With the use of Open Refine, the different spellings of **SUBWAY** can be grouped under one. 

The query now shows how **SUBWAY** is represented in the dataset *after* cleaning in Open Refine.

In [ ]:
Q = '''
SELECT estName as Name, COUNT(estName) as 'num of establishments'
FROM Establishments
WHERE Name LIKE '%subway'
GROUP BY Name
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_clean)
df.head(10)

## **VIOLATIONS**
---

In [ ]:
import pandas as pd
import numpy as np

# import dirty data
food_ins_df = pd.read_csv("../data/dirty_data/Food_Inspections.csv")
food_ins_df.columns

### **OVERVIEW**
The Violations columns contains a lot of unstructured information. There are *three* distinct parts to each violation.

1. Health Code Violation Number

2. Violation Description

3. Comments

**EXAMPLE**

    8. sanitizing rinse for equipment and utensils:  clean, proper temperature, concentration, exposure time - comments: no dish washing facilities on site, (no three compartment sink, with grease trap, or dishmachine), instructed to provide, | 11. adequate number, convenient, accessible, designed, and maintained - comments:  no exposed hand sink for rear service area, instructed to provide, | 18. no evidence of rodent or insect outer openings protected/rodent proofed, a written log shall be maintained available to the inspectors - comments: no license pest control log book at this time of inspection, instructed to provide, upon next visit

### **WEIRD CASES**
1. Sometimes violations have pretext like 'Serious Violation <number>' (row 6) or 'Critical Violation <number>' (row 6)

2. A list of old violations can appear in the comments (row 6)

#### **ASSUMPTIONS**
1. We are able to convert violations text to lowercase and lose no important meaning

2. We are able to use Regex to pull the numbers. All numbers match the pattern r'\d+\.' (number followed by period)

3. We are able to split the Violations on the number found before

4. We are able to split the violations description from comments on the string '- comments:'

##### **ASSUMPTION 1**
Not sure how to test this, looking through the data though I think this assumption is good. There doesn't seem to be any reason some things are all caps and others are not. I'm guessing these entries are copied directly from some PDF that is in all caps (lots of legal documents are).

##### **ASSUMPTION 2**

**GOOD ASSUMPTION**: using the BELOW regex, we are able to capture numbers in each row

In [ ]:
sec_assump_test = food_ins_df[['Violations']]
total_rows = sec_assump_test.shape[0]
sec_assump_test = sec_assump_test[sec_assump_test['Violations'].notna()]
print(f'There are {total_rows - sec_assump_test.shape[0]} null values for Violations')
print(f'rows left {sec_assump_test.shape[0]}')
num_reg = r'(?:^|\| )\d+\. '
contains_num = sec_assump_test['Violations'].str.contains(num_reg, regex=True)
print(f'Rows with numbers: {contains_num.sum()}')
assert sec_assump_test.shape[0] == contains_num.sum()

##### **ASSUMPTION 3**

**GOOD ASSUMPTION**: splitting on the regex creates the same number of violation numbers and violation text

In [ ]:
num_reg = r'(?:^|\| )(\d+\. )'
third_assump_test = food_ins_df[food_ins_df['Violations'].notna()][['Violations']]
nums = third_assump_test['Violations'].str.extractall(num_reg)
third_assump_test['v_num'] = nums.groupby(level=0)[0].apply(list)
third_assump_test['list_violations'] = third_assump_test['Violations'].str.split(r'(?:^|\| )\d+\. ', regex=True).apply(lambda x: x[1:])
# get counts for the lists in each row for violations and violation numbers, ensure they are the same
num_violations = third_assump_test['list_violations'].apply(lambda x: len(x))
num_violations_nums = third_assump_test['v_num'].apply(lambda x: len(x))
assert num_violations.equals(num_violations_nums)
third_assump_test[1:1000].to_csv('test.csv')

##### **ASSUMPTION 4**

**GOOD ASSUMPTION**: with the previous preprocessing, all rows have comments

In [ ]:
fourth_assump_test = third_assump_test.explode(['list_violations', 'v_num'])
fourth_assump_test['list_violations'] = fourth_assump_test['list_violations'].str.lower()
has_comments = fourth_assump_test['list_violations'].str.contains('- comments:')
assert fourth_assump_test.shape[0] == has_comments.sum()

### **CLEANED VIOLATIONS IN PRACTICE**
Below is a function that can be used to clean the violations.

In [ ]:
def clean_violations(df: pd.DataFrame): 
  df['Violations'] = df['Violations'].str.lower()
  mask = df['Violations'].notna()
  # create violation number column
  num_reg = r'(?:^|\| )(\d+\. )'
  vio_nums = df.loc[mask, 'Violations'].str.extractall(num_reg)
  df.loc[mask, 'v_num'] = vio_nums.groupby(level=0)[0].apply(list)
  # create violation_temp
  split_reg = r'(?:^|\| )\d+\. '
  df.loc[mask, 'vio_temp'] = df.loc[mask, 'Violations'].str.split(split_reg, regex=True).apply(lambda x: x[1:])
  # split into individual rows for violations
  df = df.explode(['vio_temp', 'v_num']).reset_index(drop=True)
  exp_mask = df['Violations'].notna()
  # create comment and desc violation columns
  desc_split_reg = '- comments:'
  df.loc[exp_mask, 'v_comment'] = df.loc[exp_mask, 'vio_temp'].str.split(desc_split_reg).apply(lambda x: x[-1] if len(x) > 1 else np.nan)
  df.loc[exp_mask, 'v_desc'] = df.loc[exp_mask, 'vio_temp'].str.split(desc_split_reg).apply(lambda x: x[0])
  df = df.drop(columns=['vio_temp'])
  return df

clean_vios = clean_violations(food_ins_df)
print(f'rows: {clean_vios.shape[0]}')

#### **VIOLATIONS IN THE DATABASE**
Additionally, the dirty database will show the three parts of the violation in one column; while the cleaned database will divide the three parts of the violations into three columns. 

**BEFORE**

In [ ]:
Q = '''
SELECT * 
FROM Inspections
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_dirty)
df.head(10)

**AFTER**

In [ ]:
# Violations ---> Number, Comments, Description
Q = '''
SELECT * 
FROM Inspections
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_clean)
df.head(10)

In [ ]:
db_dirty.close()
db_clean.close()